In [1]:
import requests
import pandas as pd
from pandas import json_normalize
import numpy as np
import json
import folium
from folium import plugins
from folium.plugins import HeatMap

In [2]:
# Foursquare Credentials
CLIENT_ID = "<HARD CODED CLIENT ID FOR API>"
CLIENT_SECRET = "<HARD CODED CLIENT SECRET FOR API>"
VERSION = "20210201"

# Pizza Places in Düsseldorf, North Rhine-Westphalia

In [3]:
# Set request parameters
request_params_pizza_dus = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": VERSION,
    "categoryId": "4bf58dd8d48988d1ca941735",     # categoryId is for Pizza Places
    "near": "Düsseldorf",
    "radius": 4000,
    "limit": 100
}

In [4]:
url = 'https://api.foursquare.com/v2/venues/explore'

# GET request to receive data as set in request parameters
resp = requests.get(url=url, params=request_params_pizza_dus)
data = json.loads(resp.text)

In [5]:
venues = data["response"]["groups"][0]["items"]
d = json_normalize(venues)
d.head(3)

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.postalCode,...,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.crossStreet,venue.location.neighborhood,venue.venuePage.id,flags.outsideRadius
0,e-0-4b73209cf964a520f09c2de3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4b73209cf964a520f09c2de3,Pizzeria Lupo,Bolkerstr. 52,51.226423,6.775604,"[{'label': 'display', 'lat': 51.22642250101587...",40213,...,Nordrhein-Westfalen,Deutschland,"[Bolkerstr. 52, 40213 Düsseldorf, Deutschland]","[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",0,[],NaN,NaN,NaN,NaN
1,e-0-4b45c0fcf964a520e10f26e3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4b45c0fcf964a520e10f26e3,Da Gino,Kirchfeldstr. 135,51.212267,6.784068,"[{'label': 'display', 'lat': 51.21226718188129...",40227,...,Nordrhein-Westfalen,Deutschland,"[Kirchfeldstr. 135 (Morsestr.), 40227 Düsseldo...","[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",0,[],Morsestr.,NaN,NaN,NaN
2,e-0-4d0a5cff5edd54815c3e059b-2,0,"[{'summary': 'This spot is popular', 'type': '...",4d0a5cff5edd54815c3e059b,Pizzeria Romantica,Düsselthaler Str. 48,51.233758,6.792516,"[{'label': 'display', 'lat': 51.23375784654132...",40211,...,Nordrhein-Westfalen,Deutschland,"[Düsselthaler Str. 48, 40211 Düsseldorf, Deuts...","[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",0,[],NaN,Stadt-Mitte,NaN,NaN


In [6]:
# Choose relevant columns
df = d[["venue.name", "venue.location.address", "venue.location.lat", "venue.location.lng"]]

# Rename columns
pizza_places_dus = df.rename(columns={"venue.name": "Name", "venue.location.address": "Address", "venue.location.lat": "Latitude", "venue.location.lng": "Longitude"})

# Remove entries if data==NaN and reset index
pizza_places_dus.reset_index(drop=True)
pizza_places_dus.head()

,Name,Address,Latitude,Longitude
0,Pizzeria Lupo,Bolkerstr. 52,51.226423,6.775604
1,Da Gino,Kirchfeldstr. 135,51.212267,6.784068
2,Pizzeria Romantica,Düsselthaler Str. 48,51.233758,6.792516
3,Pizzeria Pesto,Pionierstr. 54,51.214771,6.785539
4,Su Nuraghe,Am Wehrhahn 69,51.229043,6.793470


In [7]:
# Düsseldorf geolocation
lat = "51.221332448"
long = "6.785496858"

# Create Map instance für Düsseldorf
dus_map = folium.Map(location=[lat, long], zoom_start=13)
dus_map

In [8]:
# Function to add markers from df to df_map
def add_markers(df_map, df, color="red"):
    for lat, long, name in zip(df["Latitude"],
                          df["Longitude"],
                          df["Name"]):
        folium.CircleMarker(
            [lat, long],
            radius=2.5,
            fill=True,
            color=color,
            fill_color=color,
            fill_opacity=1
            ).add_to(df_map)

    display(df_map)

In [9]:
import selenium.webdriver
import time

# Function to save locally saved map (.html) as png-file
def save_map_png(filename_no_extension):
    driver = selenium.webdriver.PhantomJS()
    driver.set_window_size(1050, 850)
    driver.get(filename_no_extension+".html")
    time.sleep(2)
    driver.save_screenshot(filename_no_extension+".png")
    
    print("Success! File saved as png-file:", filename_no_extension+".png")

In [10]:
# Add pizza places to map
add_markers(dus_map, pizza_places_dus)

In [11]:
# Add HeatMap
heatmap_data = pizza_places_dus[["Latitude", "Longitude"]].to_numpy().tolist()
dus_map.add_child(plugins.HeatMap(heatmap_data))

dus_map

In [12]:
# Save map as .html and .png
dus_map.save("Map_Duesseldorf_PizzaPlaces.html")
save_map_png("Map_Duesseldorf_PizzaPlaces")

C:\Users\User\anaconda3\lib\site-packages\selenium\webdriver\phantomjs\webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


Success! File saved as png-file: Map_Duesseldorf_PizzaPlaces.png


## Colleges and Universities in Düsseldorf

In [13]:
# Set request parameters
request_params_college_dus = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": VERSION,
    "categoryId": "4d4b7105d754a06372d81259",     # categoryId is for Colleges and University
    "near": "Düsseldorf",
    "radius": 4000,
    "limit": 100
}

In [14]:
url = 'https://api.foursquare.com/v2/venues/explore'

# GET request to receive data as set in request parameters
resp = requests.get(url=url, params=request_params_college_dus)
data = json.loads(resp.text)

In [15]:
venues = data["response"]["groups"][0]["items"]
d = json_normalize(venues)
d.head(3)

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.cc,venue.location.city,...,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.address,venue.location.crossStreet,venue.location.postalCode,venue.location.neighborhood,venue.venuePage.id,flags.outsideRadius
0,e-0-4e671d91e4cdfdecb0287eae-0,0,"[{'summary': 'This spot is popular', 'type': '...",4e671d91e4cdfdecb0287eae,Walter-Eucken-Berufskolleg Dependance,51.231055,6.791989,"[{'label': 'display', 'lat': 51.23105490466366...",DE,Düsseldorf,...,"[Düsseldorf, Deutschland]","[{'id': '4d4b7105d754a06372d81259', 'name': 'C...",0,[],NaN,NaN,NaN,NaN,NaN,NaN
1,e-0-4bae0355f964a520e37a3be3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4bae0355f964a520e37a3be3,Konfuzius Institut,51.219497,6.786359,"[{'label': 'display', 'lat': 51.219497, 'lng':...",DE,Düsseldorf,...,"[Graf-Adolf Straße 63, Düsseldorf, Deutschland]","[{'id': '4bf58dd8d48988d198941735', 'name': 'C...",0,[],Graf-Adolf Straße 63,NaN,NaN,NaN,NaN,NaN
2,e-0-4c9e6c15d3c2b60c243acbbc-2,0,"[{'summary': 'This spot is popular', 'type': '...",4c9e6c15d3c2b60c243acbbc,Die Brücke am Heinrich-Heine-Platz (VHS),51.224711,6.774901,"[{'label': 'display', 'lat': 51.224711, 'lng':...",DE,Düsseldorf,...,"[Kasernenstraße 6 (At Wallstraße), 40213 Düsse...","[{'id': '4bf58dd8d48988d1a0941735', 'name': 'C...",0,[],Kasernenstraße 6,At Wallstraße,40213,NaN,NaN,NaN


In [16]:
# Choose relevant columns
df = d[["venue.name", "venue.location.address", "venue.location.lat", "venue.location.lng"]]

# Rename columns
colleges_dus = df.rename(columns={"venue.name": "Name", "venue.location.address": "Address", "venue.location.lat": "Latitude", "venue.location.lng": "Longitude"})

# Remove entries if data==NaN and reset index
colleges_dus.reset_index(drop=True)
colleges_dus.head()

,Name,Address,Latitude,Longitude
0,Walter-Eucken-Berufskolleg Dependance,NaN,51.231055,6.791989
1,Konfuzius Institut,Graf-Adolf Straße 63,51.219497,6.786359
2,Die Brücke am Heinrich-Heine-Platz (VHS),Kasernenstraße 6,51.224711,6.774901
3,Zentralbibliothek Düsseldorf,Bertha-von-Suttner-Platz 1,51.218803,6.796055
4,DAPR - Deutsche Akademie für Public Relations,NaN,51.229192,6.772290


In [17]:
add_markers(dus_map, colleges_dus, "black")

In [18]:
# Save map as .html and .png
dus_map.save("Map_Duesseldorf_PizzaPlaces_Colleges.html")
save_map_png("Map_Duesseldorf_PizzaPlaces_Colleges")

C:\Users\User\anaconda3\lib\site-packages\selenium\webdriver\phantomjs\webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


Success! File saved as png-file: Map_Duesseldorf_PizzaPlaces_Colleges.png


# Pizza Places in Essen, North Rhine-Westphalia

In [19]:
# GET request to receive data as set in request parameters
request_params = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": VERSION,
    "categoryId": "4bf58dd8d48988d1ca941735",        # categoryId is for Pizza Places
    "near": "Essen",
    "radius": 4000,
    "limit": 100
}

url = 'https://api.foursquare.com/v2/venues/explore'

# GET request to receive data as set in request parameters
resp = requests.get(url=url, params=request_params)
data = json.loads(resp.text)

In [20]:
venues = data["response"]["groups"][0]["items"]
d = json_normalize(venues)
d.head(3)

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.postalCode,...,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.crossStreet,flags.outsideRadius,venue.venuePage.id
0,e-0-4b76e544f964a52024682ee3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4b76e544f964a52024682ee3,Zodiac,Witteringstr. 41,51.441444,7.010918,"[{'label': 'display', 'lat': 51.44144354435105...",45130,...,Essen,Nordrhein-Westfalen,Deutschland,"[Witteringstr. 41, 45130 Essen, Deutschland]","[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",0,[],NaN,NaN,NaN
1,e-0-4c330dc87cc0c9b67820f19a-1,0,"[{'summary': 'This spot is popular', 'type': '...",4c330dc87cc0c9b67820f19a,Venezia,Rüttenscheider Str. 52,51.439392,7.006064,"[{'label': 'display', 'lat': 51.43939178552952...",45130,...,Essen,Nordrhein-Westfalen,Deutschland,"[Rüttenscheider Str. 52, 45130 Essen, Deutschl...","[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",0,[],NaN,NaN,NaN
2,e-0-4ccdacb6c0378cfab9f88e48-2,0,"[{'summary': 'This spot is popular', 'type': '...",4ccdacb6c0378cfab9f88e48,Il Mulino,Rathenaustr. 2,51.453595,7.011556,"[{'label': 'display', 'lat': 51.45359494950699...",45127,...,Essen,Nordrhein-Westfalen,Deutschland,"[Rathenaustr. 2, 45127 Essen, Deutschland]","[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",0,[],NaN,NaN,NaN


In [21]:
# Choose relevant columns
df = d[["venue.name", "venue.location.address", "venue.location.lat", "venue.location.lng"]]

# Rename columns
pizza_places_essen = df.rename(columns={"venue.name": "Name", "venue.location.address": "Address", "venue.location.lat": "Latitude", "venue.location.lng": "Longitude"})

# Remove entries if data==NaN and reset index
pizza_places_essen.reset_index(drop=True)
pizza_places_essen.head()

,Name,Address,Latitude,Longitude
0,Zodiac,Witteringstr. 41,51.441444,7.010918
1,Venezia,Rüttenscheider Str. 52,51.439392,7.006064
2,Il Mulino,Rathenaustr. 2,51.453595,7.011556
3,Cantina Italiana,Paulinenstr. 22,51.437926,7.012619
4,Cafe Nord,Viehofer Platz 1,51.461856,7.013737


In [22]:
# Essen geolocation
lat, long = "51.450832", "7.013056"

# Create Map instance for Essen
essen_map = folium.Map(location=[lat, long], zoom_start=13)

In [23]:
# Add markers to spot Pizza Places in Essen
add_markers(essen_map, pizza_places_essen)

In [24]:
# Add HeatMap
heatmap_data_essen = pizza_places_essen[["Latitude", "Longitude"]].to_numpy().tolist()
essen_map.add_child(plugins.HeatMap(heatmap_data_essen))

essen_map

In [25]:
# Save map as .html and .png
essen_map.save("Map_Essen_PizzaPlaces.html")
save_map_png("Map_Essen_PizzaPlaces")

C:\Users\User\anaconda3\lib\site-packages\selenium\webdriver\phantomjs\webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


Success! File saved as png-file: Map_Essen_PizzaPlaces.png


## Colleges and Universities in Essen

In [26]:
request_params_college_essen = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": VERSION,
    "categoryId": "4d4b7105d754a06372d81259",     # categoryId is for Colleges and University
    "near": "Essen",
    "radius": 4000,
    "limit": 100
}

url = 'https://api.foursquare.com/v2/venues/explore'
resp = requests.get(url=url, params=request_params_college_essen)
data = json.loads(resp.text)

In [27]:
venues = data["response"]["groups"][0]["items"]
d = json_normalize(venues)
d.head(3)

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.postalCode,...,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.neighborhood,venue.venuePage.id,flags.outsideRadius
0,e-0-4dd644bad4c05d5096cb9a7c-0,0,"[{'summary': 'This spot is popular', 'type': '...",4dd644bad4c05d5096cb9a7c,Weststadt Akademie,Thea-Leymann-Str. 35,51.457561,7.001901,"[{'label': 'display', 'lat': 51.4575614166877,...",45127,...,Essen,Nordrhein-Westfalen,Deutschland,"[Thea-Leymann-Str. 35, 45127 Essen, Deutschland]","[{'id': '4d4b7105d754a06372d81259', 'name': 'C...",0,[],NaN,NaN,NaN
1,e-0-4bc3fb93b492d13afb8da960-1,0,"[{'summary': 'This spot is popular', 'type': '...",4bc3fb93b492d13afb8da960,"Campus Essen, Schützenbahn – Universität Duisb...",Schützenbahn 70,51.461522,7.016845,"[{'label': 'display', 'lat': 51.46152226588673...",45117,...,Essen,Nordrhein-Westfalen,Deutschland,"[Schützenbahn 70, 45117 Essen, Deutschland]","[{'id': '4bf58dd8d48988d1ae941735', 'name': 'U...",0,[],NaN,NaN,NaN
2,e-0-4cc17b9f53f6b71313077df8-2,0,"[{'summary': 'This spot is popular', 'type': '...",4cc17b9f53f6b71313077df8,VWA Essen (SHZ 1),Herkulesstraße 32,51.456703,7.022815,"[{'label': 'display', 'lat': 51.45670257735735...",45127,...,Essen,Nordrhein-Westfalen,Deutschland,"[Herkulesstraße 32, 45127 Essen, Deutschland]","[{'id': '4bf58dd8d48988d1ae941735', 'name': 'U...",0,[],NaN,NaN,NaN


In [28]:
# Choose relevant columns
df = d[["venue.name", "venue.location.address", "venue.location.lat", "venue.location.lng"]]

# Rename columns
colleges_essen = df.rename(columns={"venue.name": "Name", "venue.location.address": "Address", "venue.location.lat": "Latitude", "venue.location.lng": "Longitude"})

# Remove entries if data==NaN and reset index
colleges_essen.reset_index(drop=True)
colleges_essen.head()

,Name,Address,Latitude,Longitude
0,Weststadt Akademie,Thea-Leymann-Str. 35,51.457561,7.001901
1,"Campus Essen, Schützenbahn – Universität Duisb...",Schützenbahn 70,51.461522,7.016845
2,VWA Essen (SHZ 1),Herkulesstraße 32,51.456703,7.022815
3,Universität Duisburg-Essen - Campus Essen,Universitätsstr. 2,51.463902,7.007517
4,Universität Bibliothek,NaN,51.464549,7.007665


In [29]:
add_markers(essen_map, colleges_essen, "black")

In [30]:
# Save map as .html and .png
essen_map.save("Map_Essen_PizzaPlaces_Colleges.html")
save_map_png("Map_Essen_PizzaPlaces_Colleges")

C:\Users\User\anaconda3\lib\site-packages\selenium\webdriver\phantomjs\webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


Success! File saved as png-file: Map_Essen_PizzaPlaces_Colleges.png
